##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 멀티 작업자 CPU/GPU 훈련 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/multi_worker_cpu_gpu_training"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/multi_worker_cpu_gpu_training.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

이 가이드는 TensorFlow 1에서 TensorFlow 2로 멀티 작업자 분산 훈련 워크플로를 마이그레이션하는 방법을 설명합니다.

CPU/GPU로 멀티 작업자 훈련은 다음과 같이 수행할 수 있습니다.

- TensorFlow 1에서는 일반적으로 `tf.estimator.train_and_evaluate` 및 `tf.estimator.Estimator` API를 사용합니다.
- TensorFlow 2에서는 Keras API를 사용하여 모델, 손실 함수, 옵티마이저 및 메트릭을 작성합니다. 그런 다음 `tf.distribute.experimental.ParameterServerStrategy` 또는 `tf.distribute.MultiWorkerMirroredStrategy`을 사용하여 Keras `Model.fit` API 또는 사용자 정의 훈련 루프(`tf.GradientTape` 포함)를 사용하여 여러 작업자에게 훈련을 배포합니다. 자세한 내용은 다음 튜토리얼을 참조하세요.
    - TensorFlow를 사용하여 분산 훈련하기
    - [Keras Model.fit/사용자 정의 훈련 루프를 사용한 매개변수 서버 훈련](../../tutorials/distribute/parameter_server_training.ipynb)
    - [Keras Model.fit을 사용한 MultiWorkerMirroredStrategy](../../tutorials/distribute/multi_worker_with_keras.ipynb)
    - [사용자 정의 훈련 루프가 포함된 MultiWorkerMirroredStrategy](../../tutorials/distribute/multi_worker_with_ctl.ipynb)

## 설치하기

데모를 위해 몇 가지 가져오기 및 간단한 데이터세트로 시작해 보겠습니다.

In [ ]:
# The notebook uses a dataset instance for `Model.fit` with
# `ParameterServerStrategy`, which depends on symbols in TF 2.7.
# Install a utility needed for this demonstration
!pip install portpicker

import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

TensorFlow의 멀티 머신에서 훈련을 진행하려면 `'TF_CONFIG'` 구성 환경 변수가 필요합니다. `'TF_CONFIG'`를 사용하여 `'cluster'` 및 `'task'`의 주소를 지정합니다(자세한 내용은 [Distributed_training](../...guide/distributed_training.ipynb) 가이드 참조).

In [ ]:
import json
import os

tf_config = {
    'cluster': {
        'chief': ['localhost:11111'],
        'worker': ['localhost:12345', 'localhost:23456', 'localhost:21212'],
        'ps': ['localhost:12121', 'localhost:13131'],
    },
    'task': {'type': 'chief', 'index': 0}
}

os.environ['TF_CONFIG'] = json.dumps(tf_config)

참고: 안타깝게도 TensorFlow 1은 `tf.estimator` API를 사용한 멀티 작업자 훈련에 멀티 클라이언트를 요구하므로(이 Colab 노트북에서 수행하기에는 특히 까다로울 수 있음) `'TF_CONFIG'` 환경 변수없이 노트북을 실행할 수 있게 만들어야 하며 이는 로컬 훈련을 대체하게 됩니다(자세한 내용은 [TensorFlow를 사용한 분산 훈련](../../guide/distributed_training.ipynb) 가이드의 <em data-md-type="raw_html">`'TF_CONFIG'` 환경 변수 설정하기</em> 섹션 참조).

`del` 문을 사용하여 변수를 제거합니다(단, TensorFlow 1의 실제 멀티 작업자 훈련에서는 이렇게 할 필요가 없음).

In [ ]:
del os.environ['TF_CONFIG']

## TensorFlow 1: tf.estimator API를 사용한 멀티 작업자 분산 훈련

다음 코드 조각은 TF1에서 멀티 작업자 훈련의 정식 워크플로를 보여줍니다. `tf.estimator.Estimator`, `tf.estimator.TrainSpec`, `tf.estimator.EvalSpec 및 <code data-md-type="codespan">tf.estimator.train_and_evaluate` API를 사용하여 훈련을 배포합니다.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

estimator = tf1.estimator.Estimator(model_fn=_model_fn)
train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: 배포 전략을 사용한 멀티 작업자 훈련

TensorFlow 2에서는 CPU, GPU, TPU를 사용하는 멀티 작업자 사이에서 분산 훈련이 `tf.distribute.Strategy`를 통해 수행됩니다.

다음 예제는 `tf.distribute.experimental.ParameterServerStrategy` 및 `tf.distribute.MultiWorkerMirroredStrategy`와 같은 두 가지 전략을 사용하는 방법을 보여줍니다. 이 두 전략은 멀티 작업자를 사용하는 CPU/GPU 훈련용으로 설계되었습니다.

`ParameterServerStrategy`는 *코디네이터*(`'chief'`)를 사용하여 이 Colab 노트북 환경이 낯설지 않게 만들어 줍니다. 여기에서 몇 가지 유틸리티를 사용하여 실행 가능한 환경에 필수적인 지원 요소를 설정하게 됩니다. 여기서 스레드는 매개변수 서버(`' ps'`) 및 작업자(`'worker'`) 시뮬레이션에 사용됩니다. 매개변수 서버 훈련에 대한 자세한 내용은 [ParameterServerStrategy를 사용한 매개변수 서버 훈련](../../tutorials/distribute/parameter_server_training.ipynb) 튜토리얼을 참조하세요.

이 예제에서는 먼저 `tf.distribute.cluster_resolver.TFConfigClusterResolver`를 사용하여 `'TF_CONFIG'` 환경 변수를 정의하여 클러스터 정보를 제공합니다. 분산 훈련에 클러스터 관리 시스템을 사용하는 경우 이미 `'TF_CONFIG'`를 제공하는지 확인합니다. 이 경우 이 환경 변수를 명시적으로 설정할 필요가 없습니다(자세한 내용은 [TensorFlow를 사용한 분산 훈련](../../guide/distributed_training.ipynb) 가이드의 <em data-md-type="raw_html">`'TF_CONFIG'` 환경 변수 설정하기</em> 섹션 참조).

In [ ]:
# Find ports that are available for the `'chief'` (the coordinator),
# `'worker'`s, and `'ps'` (parameter servers).
import portpicker

chief_port = portpicker.pick_unused_port()
worker_ports = [portpicker.pick_unused_port() for _ in range(3)]
ps_ports = [portpicker.pick_unused_port() for _ in range(2)]

# Dump the cluster information to `'TF_CONFIG'`.
tf_config = {
    'cluster': {
        'chief': ["localhost:%s" % chief_port],
        'worker': ["localhost:%s" % port for port in worker_ports],
        'ps':  ["localhost:%s" % port for port in ps_ports],
    },
    'task': {'type': 'chief', 'index': 0}
}
os.environ['TF_CONFIG'] = json.dumps(tf_config)

# Use a cluster resolver to bridge the information to the strategy created below.
cluster_resolver = tf.distribute.cluster_resolver.TFConfigClusterResolver()

그런 다음 작업자 및 매개변수 서버에 대해 `tf.distribute.Server`를 하나씩 생성합니다.

In [ ]:
# Workers need some inter_ops threads to work properly.
# This is only needed for this notebook to demo. Real servers
# should not need this.
worker_config = tf.compat.v1.ConfigProto()
worker_config.inter_op_parallelism_threads = 4

for i in range(3):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="worker",
      task_index=i,
      config=worker_config)

for i in range(2):
  tf.distribute.Server(
      cluster_resolver.cluster_spec(),
      job_name="ps",
      task_index=i)

실제 분산 훈련에서는 코디네이터에서 모든 `tf.distribute.Server`를 시작하는 대신 여러 머신을 사용하고, `"worker"` 및 `"ps"`(매개변수 서버)로 지정된 머신은 각각 `tf.distribute.Server`를 실행합니다. 자세한 내용은 [매개변수 서버 훈련](../../tutorials/distribute/parameter_server_training.ipynb) 튜토리얼의 *실제 클러스터* 섹션을 참조하세요.

모든 것이 준비되면 `ParameterServerStrategy` 객체를 생성합니다.

In [ ]:
strategy = tf.distribute.experimental.ParameterServerStrategy(cluster_resolver)

전략 객체를 생성한 후에는 모델, 옵티마이저 및 기타 변수를 정의하고 `Strategy.scope` API 내에서 Keras `Model.compile`을 호출하여 훈련을 배포합니다.(자세한 내용은 `Strategy.scope` API 문서 참조).

예를 들어, 순방향 및 역방향 전달을 정의하여 훈련을 사용자 정의하려면 [매개변수 서버 훈련](../../tutorials/distribute/parameter_server_training.ipynb) 튜토리얼의 *사용자 정의 훈련 루프를 사용한 훈련* 섹션을 참조하세요.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(
      (features, labels)).shuffle(10).repeat().batch(64)

eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).repeat().batch(1)

with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.legacy.Adagrad(learning_rate=0.05)
  model.compile(optimizer, "mse")

model.fit(dataset, epochs=5, steps_per_epoch=10)

In [ ]:
model.evaluate(eval_dataset, steps=10, return_dict=True)

> **파티셔너(`tf.distribute.experimental.partitioners`)**
>
> TensorFlow 2의 `ParameterServerStrategy`는 변수 파티셔닝을 지원하고 TensorFlow 1과 동일한 파티셔너를 덜 혼란스러운 이름으로 제공합니다.
>
> - `tf.compat.v1.variable_axis_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.MaxSizePartitioner`: 샤드를 최대 크기 미만으로 유지하는 파티셔너입니다.
> - `tf.compat.v1.min_max_variable_partitioner` -&gt; `tf.distribute.experimental.partitioners.MinSizePartitioner`: 샤드당 최소 크기를 할당하는 파티셔너입니다.
> - `tf.compat.v1.fixed_size_partitioner` -&gt; `tf.distribute.experimental.partitioners.FixedShardsPartitioner`: 고정된 수의 샤드를 할당하는 파티셔너입니다.

또는, `MultiWorkerMirroredStrategy` 객체를 사용할 수 있습니다.

In [ ]:
# To clean up the `TF_CONFIG` used for `ParameterServerStrategy`.
del os.environ['TF_CONFIG']
strategy = tf.distribute.MultiWorkerMirroredStrategy()

위에서 사용한 전략을 `MultiWorkerMirroredStrategy` 객체로 대체하고 이 전략으로 훈련을 수행할 수 있습니다.

`tf.estimator` API와 마찬가지로 `MultiWorkerMirroredStrategy`는 멀티 클라이언트 전략이므로 이 Colab 노트북에서 분산 훈련을 쉽게 실행할 수 있는 방법은 없습니다. 따라서 위의 코드를 이 전략으로 바꾸면 결국 로컬에서 실행됩니다. [Keras Model.fit](../../tutorials/distribute/multi_worker_with_keras.ipynb)/[사용자 정의 훈련 루프](../../tutorials/distribute/multi_worker_with_ctl.ipynb)를 사용한 멀티 작업자 훈련 튜토리얼은 Colab의 로컬 호스트에 두 명의 작업자가 있을 때 `'TF_CONFIG'` 변수로 멀티 작업자 훈련을 실행하는 방법을 보여줍니다. 실제로는 외부 IP 주소/포트에 여러 작업자를 생성하고 `'TF_CONFIG'` 변수를 사용하여 각 작업자의 클러스터 구성을 지정하게 될 수도 있습니다.

## 다음 단계

TensorFlow 2의 `tf.distribute.experimental.ParameterServerStrategy` 및 `tf.distribute.MultiWorkerMirroredStrategy`를 사용한 멀티 작업자 분산 훈련에 대한 자세한 내용은 다음 리소스를 참조하세요.

- 튜토리얼: [ParameterServerStrategy 및 Keras Model.fit/a 사용자 정의 훈련 루프를 사용한 매개변수 서버 훈련](../../tutorials/distribute/parameter_server_training.ipynb)
- 튜토리얼: [MultiWorkerMirroredStrategy 및 Keras Model.fit을 사용한 멀티 작업자 훈련](../../tutorials/distribute/multi_worker_with_keras.ipynb)
- 튜토리얼: [MultiWorkerMirroredStrategy 및 사용자 정의 훈련 루프를 사용한 멀티 작업자 훈련](../../tutorials/distribute/multi_worker_with_ctl.ipynb)
- 가이드: [TensorFlow를 사용하여 분산 훈련하기](../../guide/distributed_training.ipynb)
- 가이드: [TensorFlow Profiler로 TensorFlow GPU 성능 최적화하기](../../guide/gpu_performance_analysis.ipynb)
- 가이드: [GPU 사용](../../guide/gpu.ipynb)(여러 GPU 사용하기 섹션)